In [1]:
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html 
# best explanation about Sequence-to-Sequence

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import pandas as pd
excel_1 = pd.read_excel("1_구어체(1).xlsx")
excel_2 = pd.read_excel("1_구어체(2).xlsx")
excel_result = excel_1

excel_result.to_csv('kor-eng.txt', sep='\t', index=False)

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else: 
            self.word2count[word] += 1
        

In [4]:
def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^A-Za-z0-9가-힣.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else: 
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [12]:
MAX_LENGTH = 15
'''
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
'''

kor_prefixes= (
    "나는", "난",
    "그는", "그녀는",
    "우리는", "그들은",
    "너는", "넌",
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[0].startswith(kor_prefixes)
    # and p[1].startswith(eng_prefixes)
    # 만약 reverse=False면 p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [13]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [14]:
input_lang, output_lang, pairs = prepareData('kor', 'eng', False)
print(random.choice(pairs))

Reading lines...
Read 199999 sentence pairs
Trimmed to 28557 sentence pairs
Counting words...
Counted words:
kor 35037
eng 12080
['그녀는 자기 사촌 결혼식 때문에 서울에 왔습니다 .', 'She came to Seoul for her cousin s wedding .']


In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [16]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        # what does .view(1, 1, -1) mean?
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [17]:
# class AttnDecoderRNN(nn.Module):

In [18]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # Why append the EOS tokens for both input/output tensor sequence
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [19]:
teacher_forcing_ratio = 1

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden 
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
        
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    

In [20]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [21]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters+1): 
        training_pair = training_pairs[iter-1]
        input_tensor  = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) loss: %.4f' % (timeSince(start, iter / n_iters), iter, (iter / n_iters * 100), print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [22]:
# need to draw plot

In [23]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 42s (- 24m 0s) (5000 6%) 4.8065
3m 15s (- 21m 11s) (10000 13%) 4.1668
4m 47s (- 19m 8s) (15000 20%) 3.9436
6m 17s (- 17m 18s) (20000 26%) 3.7638
7m 48s (- 15m 36s) (25000 33%) 3.6191
9m 19s (- 13m 59s) (30000 40%) 3.4802
10m 50s (- 12m 23s) (35000 46%) 3.4111
12m 21s (- 10m 48s) (40000 53%) 3.3203
13m 40s (- 9m 7s) (45000 60%) 3.2371
15m 2s (- 7m 31s) (50000 66%) 3.1467
16m 24s (- 5m 58s) (55000 73%) 3.0784
17m 45s (- 4m 26s) (60000 80%) 2.9846
19m 7s (- 2m 56s) (65000 86%) 2.9486
20m 29s (- 1m 27s) (70000 93%) 2.8695
21m 50s (- 0m 0s) (75000 100%) 2.8324


In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            # decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words # decoder_attentions[:di + 1]
    
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

def translate(sentence):
    output_words = evaluate(encoder1, decoder1, sentence)
    output_sentence = ' '.join(output_words)
    print("input(kor): " + sentence)
    print("output(eng): " + output_sentence + "\n")

In [25]:
evaluateRandomly(encoder1, decoder1)

> 나는 하나님이 사용하시는 사람이에요 .
= I am a man who is used by God .
< I am a man who is a man who is the most precious person .

> 나는 마음속으로 그 친구를 미워하였기 때문에 .
= It s because I hated him in my heart .
< I was happy because I was a friend . <EOS>

> 나는 김동인이 쓴 감자라는 책을 읽었어 .
= I read a book called Potato written by Dongin Kim .
< I read a book called The book . <EOS>

> 우리는 김포공항에서 출발해야 해요 .
= We have to depart from Kimpo Airport .
< We have to get out of here . <EOS>

> 나는 샘플을 빨리 받을 수 있기를 바랍니다 .
= I hope to receive samples soon .
< I want to go to the party . <EOS>

> 나는 군대에서 전역했습니다 .
= I retired from the army .
< I m a high teacher . <EOS>

> 난 매주 교회에 가서 항상 널 위해 기도할게 .
= I ll go to church every week and always pray for you .
< I always go to the party for the first time . <EOS>

> 나는 아무 책임도 없습니다 .
= I m not responsible at all .
< I don t have a job . <EOS>

> 그녀는 오후에 카페에서 커피를 마셔 .
= In the afternoon she drinks the coffee at the coffee shop .
< She has been to the Philippines to get out of the first 

In [27]:
translate("저는 오늘 시장에 갑니다")
translate("그는 많이 아픕니다")
translate("그녀는 저녁을 먹고 싶습니다")
translate("나는 많은 것을 배울 수 있습니다")
translate("너는 나를 사랑합니다")
translate("저는 방에 누가 있는지 모릅니다")

input(kor): 저는 오늘 시장에 갑니다
output(eng): I will go to the party by tomorrow morning . <EOS>

input(kor): 그는 많이 아픕니다
output(eng): He is a very famous . <EOS>

input(kor): 그녀는 저녁을 먹고 싶습니다
output(eng): She is really good at first sight . <EOS>

input(kor): 나는 많은 것을 배울 수 있습니다
output(eng): I can make a lot of things . <EOS>

input(kor): 너는 나를 사랑합니다
output(eng): You know who love me . <EOS>

input(kor): 저는 방에 누가 있는지 모릅니다
output(eng): We will make a contract with the customer that you sent us . <EOS>

